Setup

In [43]:
import pandas as pd
import numpy as np
import sqlalchemy as db
import matplotlib.pyplot as plt
import seaborn as sns

# Koneksi ke database SQLite
database = db.create_engine("sqlite:///database_HIV_AIDS.sqlite")
conn = database.connect()

Menampilkan beberapa sampel dari ketiga tabel data:

In [44]:
deaths_df = pd.read_sql("data_deaths_persons_diagnosed", conn)
living_df = pd.read_sql("data_persons_living", conn)
newly_diagnosed_df = pd.read_sql("data_persons_newly_diagnosed", conn)

deaths_df
living_df
newly_diagnosed_df

,Year,Category,Group,Count
0,2010,Age at Diagnosis (in years),0 to 11,16
1,2010,Age at Diagnosis (in years),12 to 14,3
2,2010,Age at Diagnosis (in years),15 to 17,30
3,2010,Age at Diagnosis (in years),18 to 19,131
4,2010,Age at Diagnosis (in years),20 to 24,780
...,...,...,...,...
1717,2022,City of Residence at Diagnosis,Long Beach,90
1718,2022,City of Residence at Diagnosis,Pasadena,10
1719,2023,City of Residence at Diagnosis,Berkeley,9
1720,2023,City of Residence at Diagnosis,Long Beach,98


2.1 Identifikasi korban penyakit HIV AIDS tertinggi dari data itu apakah laki - laki atau perempuan

In [45]:
korban_HIV = """
SELECT * FROM data_deaths_persons_diagnosed WHERE Category = 'Current Gender';
"""

data_korban_HIV = pd.read_sql_query(korban_HIV, conn)
data_korban_HIV

,Year,Category,Group,Count
0,2010,Current Gender,Cisgender men,1414
1,2010,Current Gender,Cisgender women,238
2,2010,Current Gender,Trans women,19
3,2010,Current Gender,Trans men,0
4,2010,Current Gender,Alternative gender,0
...,...,...,...,...
65,2023,Current Gender,Cisgender men,1900
66,2023,Current Gender,Cisgender women,298
67,2023,Current Gender,Trans women,54
68,2023,Current Gender,Trans men,0


In [41]:
korban_HIV_gender = """
SELECT
    (SELECT SUM(Count)
     FROM data_deaths_persons_diagnosed
     WHERE "Group" LIKE '% women') AS korban_HIV_women,

    (SELECT SUM(Count)
     FROM data_deaths_persons_diagnosed
     WHERE "Group" LIKE '% men') AS korban_HIV_men
"""
data_korban_HIV_gender = pd.read_sql_query(korban_HIV_gender, conn)
data_korban_HIV_gender


,korban_HIV_women,korban_HIV_men
0,3889,22797


2.2 Apakah Wanita Paling Rentan di Semua Kategori

In [42]:
korban_HIV_gender_kategori = """
WITH combined_data AS (
    SELECT
        year        AS Year,
        'Living'    AS Dataset,
        "Group",
        Count
    FROM data_persons_living
    WHERE Category = 'Current Gender'

    UNION ALL

    SELECT
        year        AS Year,
        'Newly Diagnosed' AS Dataset,
        "Group",
        Count
    FROM data_persons_newly_diagnosed
    WHERE Category = 'Current Gender'

    UNION ALL

    SELECT
        year        AS Year,
        'Deaths'    AS Dataset,
        "Group",
        Count
    FROM data_deaths_persons_diagnosed
    WHERE Category = 'Current Gender'
)
SELECT
    Dataset,
    "Group",
    SUM(Count) AS Total_Count
FROM combined_data
GROUP BY Dataset, "Group"
ORDER BY Dataset, Total_Count DESC
"""

data_korban_gender_kategori = pd.read_sql_query(korban_HIV_gender_kategori, conn)
data_korban_gender_kategori


,Dataset,Group,Total_Count
0,Deaths,Cisgender men,22788
1,Deaths,Cisgender women,3470
2,Deaths,Trans women,419
3,Deaths,Trans men,9
4,Deaths,Alternative gender,1
5,Living,Cisgender men,1593892
6,Living,Cisgender women,217242
7,Living,Trans women,32461
8,Living,Trans men,966
9,Living,Alternative gender,160


Dari atas, yang menjadi paling rentan di semua kategori adalah men.